![alt text](assets/shanaly_Ai.jpg)"

In [1]:
# import library 

import torch
import sys
import os
import torchvision
import tqdm

In [2]:
# Print library version 
print(sys.platform)
print(f"torch version is {torch.__version__}")
print(f"torchvision version {torchvision.__version__}")
print(f"tqdm version {tqdm.__version__}")

win32
torch version is 2.5.1+cpu
torchvision version 0.20.1+cpu
tqdm version 4.67.0


In [14]:
# import class and module 
import torch.nn as nn
import torch.utils.data as Data
from torchvision import transforms, datasets
from tqdm.notebook import tqdm
import torch.nn.functional as F


In [4]:
# load data from folders 
train_data_dir = os.path.join('data', 'train')
val_data_dir = os.path.join('data', 'val')

In [5]:
class ConvertToRGB:
    def __call__(self, image):
        if image.mode != 'RGB':
            image = image.convert('RGB')
        return image

In [6]:
transform = transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((225, 225)),
    transforms.ToTensor()
])

In [7]:
# establish dataset 
train_dataset = datasets.ImageFolder(train_data_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_data_dir, transform=transform)

In [121]:
val_dataset

Dataset ImageFolder
    Number of datapoints: 153
    Root location: data\val
    StandardTransform
Transform: Compose(
               Resize(size=(225, 225), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [8]:
# made a dataloader 
train_loader = Data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = Data.DataLoader(val_dataset, batch_size=8, shuffle=True)

In [25]:
# cnn model 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 64, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()

        self.out = nn.Linear(64 * 28 * 28, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = self.flatten(x)
        output = self.out(x)
        return output

In [26]:
model = CNN()
print(model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (out): Linear(in_features=50176, out_features=2, bias=True)
)


In [27]:
# hyperparameter 


BATCH_SIZE = 8
Lr = 0.001

optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
criterion = nn.CrossEntropyLoss()

In [28]:
def train(model, dataloader):
    model.train()
    total_loss = 0.0
    for img, target in tqdm(dataloader, desc='TRAINING', leave=False):

        optimizer.zero_grad
        output = model(img)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Loss {total_loss/ len(dataloader.dataset)}")

In [35]:
train(model, train_loader)

TRAINING:   0%|          | 0/31 [00:00<?, ?it/s]

Loss 0.08810159246452519


In [30]:
def predict(model, data_loader, device="cpu"):
    all_probs = torch.tensor([]).to(device)
    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, desc="Predicting", leave=False):
            inputs = inputs.to(device)
            output = model(inputs)
            probs = F.softmax(output, dim=1)
            all_probs = torch.cat((all_probs, probs), dim=0)

    return all_probs

In [31]:
probabilities_train = predict(model, train_loader)
probabilities_train.shape

Predicting:   0%|          | 0/31 [00:00<?, ?it/s]

torch.Size([244, 2])

In [32]:
def score(model, data_loader, loss_fn, device="cpu"):
    
    total_loss = 0
    total_correct = 0

    model.eval()

  
    with torch.no_grad():
        
        for inputs, targets in tqdm(data_loader, desc="Scoring", leave=False):
            inputs = inputs.to(device)
            output = model(inputs)

            targets = targets.to(device)
            loss = loss_fn(output, targets)
            total_loss += loss.data.item() * inputs.size(0)

            correct = torch.eq(torch.argmax(output, dim=1), targets)
            total_correct += torch.sum(correct).item()

    return total_loss / len(data_loader.dataset), total_correct / len(
        data_loader.dataset
    )

In [33]:
loss_train, accuracy_train = score(model, train_loader, criterion)
print(f"Training accuracy from score function: {accuracy_train} and loss is {loss_train:.2f}")

Scoring:   0%|          | 0/31 [00:00<?, ?it/s]

Training accuracy from score function: 0.5 and loss is 0.69


In [36]:
loss_train, accuracy_train = score(model, val_loader, criterion)
print(f"Training accuracy from score function: {accuracy_train} and loss is {loss_train:.2f}")

Scoring:   0%|          | 0/20 [00:00<?, ?it/s]

Training accuracy from score function: 0.5882352941176471 and loss is 0.69
